In [ ]:
import itertools
from carbonplan_benchmarks.utils import shuffle_runs

In [ ]:
attrs = {
    "variable": "tasmax",
    "model": "ACCESS-CM2",
    "data_type": "pyramids",
    "variant": "r1i1p1f1",
    "zarr_version": "v2",
    "shard_orientation": "0",
    "data_shard_size": "0",
    "data_dtype": "f4",
    "coordinate_chunks": "0",
    "coordinate_shards": "0",
    "compression": "gzipL1",
    "write_empty_chunks": True,
    "inflevel": "100",
}
pixels_per_tile = [128, 256]
data_chunk_size = [1, 5, 10, 25]
projections = ["3857", "4326"]

zarr_version = "v3"
data_shard_size = [0, 50, 100]
shard_orientation = ["both"]

groups = ['0', '1', '2', '3']
sharded_arrays = ['tasmax']
arrays = ['tasmax', 'time', 'x', 'y']

In [ ]:
datasets = []
for pix, chunk, proj in itertools.product(pixels_per_tile, data_chunk_size, projections):
    path = f"{attrs['data_type']}-{attrs['zarr_version']}-{proj}-{attrs['write_empty_chunks']}-{pix}-{chunk}-{attrs['shard_orientation']}-{attrs['data_shard_size']}-{attrs['data_dtype']}-{attrs['data_shard_size']}-{attrs['coordinate_chunks']}-{attrs['coordinate_shards']}-{attrs['compression']}-{attrs['inflevel']}"
    datasets.append(path)
    if proj == "3857":
        projection = "mercator"
    else:
        projection = "equirectangular"
    print("{")
    print(
        f"\t...SHARED_ATTRS,\n\tid: \'{path}\',\n\t"
        f"version: 'v2',\n\t"
        f"chunkSize: {chunk},\n\t"
        f"timeChunks: 10,\n\t"
        f"projection: '{projection}',\n\t"
        f"consolidatedMetadata: true,\n\t"
        f"shardOrientation: false,\n\t"
        f"shardSize: false,\n\t"
    )
    print("},")
for pix, chunk, proj, shard, orient in itertools.product(
    pixels_per_tile, data_chunk_size, projections, data_shard_size, shard_orientation
):
    if shard == 0:
        orient = 0
    path = f"{attrs['data_type']}-{zarr_version}-{proj}-{attrs['write_empty_chunks']}-{pix}-{chunk}-{orient}-{shard}-{attrs['data_dtype']}-{attrs['coordinate_chunks']}-{attrs['coordinate_shards']}-{attrs['compression']}-{attrs['inflevel']}"
    datasets.append(path)
    if proj == "3857":
        projection = "mercator"
    else:
        projection = "equirectangular"
    if shard == 0:
        orient = "false"
    print("{")
    print(
        f"\t...SHARED_ATTRS,\n\tid: \'{path}\',\n\t"
        f"version: 'v3',\n\t"
        f"chunkSize: {chunk},\n\t"
        f"timeChunks: 10,\n\t"
        f"projection: '{projection}',\n\t"
        f"consolidatedMetadata: false,\n\t"
        f"shardOrientation: '{orient}',\n\t"
        f"shardSize: {shard},\n\t"
    )
    print("},")

In [ ]:
commands = shuffle_runs(datasets=datasets, nruns=1, timeout=5000, detect_provider=True)
for command in commands:
    print(command)